In [80]:
from pynq import Overlay
from pynq import allocate
import numpy as np

In [ ]:
# Path menuju file input dan file weight dan bias

dos_wb = '/home/xilinx/vae_inputs/input_buffer_dos.txt'
port_wb = '/home/xilinx/vae_inputs/input_buffer_port.txt'
ddos_wb = '/home/xilinx/vae_inputs/input_buffer_ddos.txt'
patator_wb = '/home/xilinx/vae_inputs/input_buffer_patator.txt'

# dos_in = '/home/xilinx/vae_inputs/train_benign_for_dos_hex.txt'
# port_in = '/home/xilinx/vae_inputs/train_benign_for_port_hex.txt'
# ddos_in = '/home/xilinx/vae_inputs/train_benign_for_ddos_hex.txt'
# patator_in = '/home/xilinx/vae_inputs/train_benign_for_patator_hex.txt'

dos_in = '/home/xilinx/vae_inputs/mix_benign_dos.txt'
port_in = '/home/xilinx/vae_inputs/mix_benign_port.txt'
ddos_in = '/home/xilinx/vae_inputs/mix_benign_ddos.txt'
patator_in = '/home/xilinx/vae_inputs/mix_benign_patator.txt'

In [82]:
# Membuat array untuk input_buffer VAE (I)

def load_file1(file1_path):
    """Membaca file pertama dan menyimpannya sebagai list berisi 27 elemen."""
    with open(file1_path, 'r') as f:
        file1_data = [line.strip() for line in f.readlines()]
    return file1_data


def load_file2(file2_path, max_lines=50):
    """Membaca file kedua dan mengembalikan daftar dari daftar (list of lists), dibatasi max_lines baris."""
    with open(file2_path, 'r') as f:
        file2_data = [line.strip().split(', ') for line in f.readlines()[:max_lines]]
    return file2_data


def combine_files(file1_data, file2_data):
    """Menggabungkan data dari file pertama dengan setiap baris dari file kedua."""
    buffer_n = []
    for row in file2_data:
        buffer_n.append(file1_data + row)  # Gabungkan 27 elemen dari file1 dan 9 dari file2
    return buffer_n

def print_formatted_output(buffer_n):
    """Mencetak output dengan format yang lebih mudah dibaca."""
    for i, row in enumerate(buffer_n):
        print(f"Array {i + 1}:")
        for j in range(0, len(row), 9):  # Cetak dalam kelompok 9 elemen
            print(" ".join(row[j:j+9]))
        print()
        
def load_file3(file1_path):
    """Membaca file pertama dan menyimpannya sebagai list berisi 27 elemen."""
    with open(file1_path, 'r') as f:
        file3_data = [line.strip() for line in f.readlines()[:10]]
    return file3_data

def fixed_point_to_decimal(value, integer_bits=5, fractional_bits=10):
    """
    Mengonversi bilangan biner/heksadesimal dalam format Q5.10 ke desimal.

    Args:
        value (str): Nilai dalam format biner (16-bit) atau heksadesimal (4-digit).
        integer_bits (int): Jumlah bit untuk bagian integer.
        fractional_bits (int): Jumlah bit untuk bagian fraksional.

    Returns:
        float: Nilai desimal yang dikonversi dari fixed-point.
    """
    # Total bit dalam representasi fixed-point
    total_bits = 1 + integer_bits + fractional_bits  # Total 16 bit untuk Q5.10

    # Jika input dalam format heksadesimal, konversi ke bilangan bulat
    if isinstance(value, str) and len(value) == 4:  # Heksadesimal (misal: '000C')
        fixed_point_value = int(value, 16)
    elif isinstance(value, str) and len(value) == 16:  # Biner (misal: '0000000000001100')
        fixed_point_value = int(value, 2)
    else:
        raise ValueError("Input harus berupa string biner (16-bit) atau heksadesimal (4-digit).")

    # Jika bilangan dalam two's complement (negatif)
    if fixed_point_value >= (1 << (total_bits - 1)):  # Cek bit tanda (bit MSB)
        fixed_point_value -= (1 << total_bits)  # Konversi dari two's complement

    # Konversi ke desimal
    scale_factor = 2 ** fractional_bits
    decimal_value = fixed_point_value / scale_factor

    return decimal_value

In [83]:
# Membuat array untuk input_buffer VAE (II)

file1_data_dos = load_file1(dos_wb)
file2_data_dos = load_file2(dos_in, max_lines=2000)
buffer_n_dos = combine_files(file1_data_dos, file2_data_dos)

file1_data_port = load_file1(port_wb)
file2_data_port = load_file2(port_in, max_lines=2000)
buffer_n_port = combine_files(file1_data_port, file2_data_port)

file1_data_ddos = load_file1(ddos_wb)
file2_data_ddos = load_file2(ddos_in, max_lines=4000)
buffer_n_ddos = combine_files(file1_data_ddos, file2_data_ddos)

file1_data_patator = load_file1(patator_wb)
file2_data_patator = load_file2(patator_in, max_lines=2000)
buffer_n_patator = combine_files(file1_data_patator, file2_data_patator)

print("dos:", len(buffer_n_dos), "\n")
# print(buffer_n_dos)
print("portscan:", len(buffer_n_port), "\n")
print("ddos:", len(buffer_n_ddos), "\n")
print("patator:", len(buffer_n_patator), "\n")

dos: 2000 

portscan: 2000 

ddos: 4000 

patator: 2000 



In [84]:
# Program bitstream to FPGA
overlay = Overlay('/home/xilinx/design_13.bit')

In [85]:
# Access to AXI DMA
dma_0 = overlay.axi_dma_0
dma_1 = overlay.axi_dma_1 
dma_2 = overlay.axi_dma_2
dma_3 = overlay.axi_dma_3

# Access send and receive channels for each DMA
dma_0_send = dma_0.sendchannel
dma_0_recv = dma_0.recvchannel

dma_1_send = dma_1.sendchannel
dma_1_recv = dma_1.recvchannel

dma_2_send = dma_2.sendchannel
dma_2_recv = dma_2.recvchannel

dma_3_send = dma_3.sendchannel
dma_3_recv = dma_3.recvchannel

In [86]:
# Allocate buffers for each DMA
input_buffer_0 = allocate(shape=(36,), dtype=np.uint64)
output_buffer_0 = allocate(shape=(9,), dtype=np.uint64)

input_buffer_1 = allocate(shape=(36,), dtype=np.uint64)
output_buffer_1 = allocate(shape=(9,), dtype=np.uint64)

input_buffer_2 = allocate(shape=(36,), dtype=np.uint64)
output_buffer_2 = allocate(shape=(9,), dtype=np.uint64)

input_buffer_3 = allocate(shape=(36,), dtype=np.uint64)
output_buffer_3 = allocate(shape=(9,), dtype=np.uint64)

In [87]:
# Inisialisasi buffer_n sebagai list kosong
buffer_to_ae_dos = []
buffer_to_ae_port = []
buffer_to_ae_ddos = []
buffer_to_ae_patator = []

In [88]:
# Memanggil hardware vae

for i, row in enumerate(buffer_n_dos):
    # Konversi dari string heksadesimal ke uint64
    input_buffer_0[:] = [int(value, 16) for value in row]
    dma_0_send.transfer(input_buffer_0)
    dma_0_send.wait()
    dma_0_recv.transfer(output_buffer_0)
    dma_0_recv.wait()
    buffer_to_ae_dos.append(output_buffer_0.copy()) 
#     total_output_dos = np.sum(output_buffer_0)
#     detected_dos[i] = 1 if total_output_dos > 0.2 else 0
 
buffer_to_ae_dos = np.array(buffer_to_ae_dos, dtype=np.uint64)
# print("Detected dos array:", detected_dos)
# print(buffer_to_ae_dos[1005:1015])

for i, row in enumerate(buffer_n_port):
    # Konversi dari string heksadesimal ke uint64
    input_buffer_1[:] = [int(value, 16) for value in row]
    dma_1_send.transfer(input_buffer_1)
    dma_1_send.wait()
    dma_1_recv.transfer(output_buffer_1)
    dma_1_recv.wait()
    buffer_to_ae_port.append(output_buffer_1.copy()) 
#     total_output_port = np.sum(output_buffer_1)
#     detected_port[i] = 1 if total_output_port > 0.2 else 0
    
buffer_to_ae_port = np.array(buffer_to_ae_port, dtype=np.uint64)
# print("Detected port array:", detected_port)
# print(buffer_to_ae_port[1005:1015])

for i, row in enumerate(buffer_n_ddos):
    # Konversi dari string heksadesimal ke uint64
    input_buffer_2[:] = [int(value, 16) for value in row]
    dma_2_send.transfer(input_buffer_2)
    dma_2_send.wait()
    dma_2_recv.transfer(output_buffer_2)
    dma_2_recv.wait()
    buffer_to_ae_ddos.append(output_buffer_2.copy())
#     total_output_ddos = np.sum(output_buffer_2)
#     detected_ddos[i] = 1 if total_output_ddos > 0.8 else 0
    
buffer_to_ae_ddos = np.array(buffer_to_ae_ddos, dtype=np.uint64)
# print("Detected ddos array:", detected_ddos)
# print(buffer_to_ae_ddos[1005:1015])

for i, row in enumerate(buffer_n_patator):
    # Konversi dari string heksadesimal ke uint64
    input_buffer_3[:] = [int(value, 16) for value in row]
    dma_3_send.transfer(input_buffer_3)
    dma_3_send.wait()
    dma_3_recv.transfer(output_buffer_3)
    dma_3_recv.wait()
    buffer_to_ae_patator.append(output_buffer_3.copy())
#     total_output_patator = np.sum(output_buffer_3)
#     detected_patator[i] = 1 if total_output_patator > 0.15 else 0

buffer_to_ae_patator = np.array(buffer_to_ae_patator, dtype=np.uint64)
# print("Detected patator array:", detected_patator)
# print(buffer_to_ae_patator[1005:1015])
    

In [89]:
# Simpan output vae agar bisa jadi input classifier

# AE dos
msb_buffer_ae_dos = buffer_to_ae_dos >> np.uint64(48)  # Geser 48 bit ke kanan
hex_buffer_ae_dos = np.vectorize(lambda x: "0x%016X" % x)(msb_buffer_ae_dos)
# print(hex_buffer_ae_dos)

# AE portscan
msb_buffer_ae_port = buffer_to_ae_port >> np.uint64(48)  # Geser 48 bit ke kanan
hex_buffer_ae_port = np.vectorize(lambda x: "0x%016X" % x)(msb_buffer_ae_port)
# print(hex_buffer_ae_port)

# AE ddos
msb_buffer_ae_ddos = buffer_to_ae_ddos >> np.uint64(48)  # Geser 48 bit ke kanan
hex_buffer_ae_ddos = np.vectorize(lambda x: "0x%016X" % x)(msb_buffer_ae_ddos)
# print(hex_buffer_ae_ddos)

# AE patator
msb_buffer_ae_patator = buffer_to_ae_patator >> np.uint64(48)  # Geser 48 bit ke kanan
hex_buffer_ae_patator = np.vectorize(lambda x: "0x%016X" % x)(msb_buffer_ae_patator)
# print(hex_buffer_ae_patator)


In [90]:
# Free buffers

del input_buffer_0
del output_buffer_0

del input_buffer_1
del output_buffer_1

del input_buffer_2
del output_buffer_2

del input_buffer_3
del output_buffer_3

# Yang terpakai hex_buffer_n
del buffer_to_ae_dos 
del buffer_to_ae_port
del buffer_to_ae_ddos
del buffer_to_ae_patator

del overlay  # Menghapus overlay sebelumnya

In [91]:
# Path menuju file weight dan bias classifier
ae_dos_wb = '/home/xilinx/vae_inputs/ae_wb_dos.txt'
ae_port_wb = '/home/xilinx/vae_inputs/ae_wb_port.txt'
ae_ddos_wb = '/home/xilinx/vae_inputs/ae_wb_ddos.txt'
ae_patator_wb = '/home/xilinx/vae_inputs/ae_wb_patator.txt'

In [92]:
# Program classifier bitstream to FPGA
overlay = Overlay('/home/xilinx/vae_inputs/design_29.bit')

In [93]:
# Access to AXI DMA
dma_0 = overlay.axi_dma_0
# dma_1 = overlay.axi_dma_1 
# dma_2 = overlay.axi_dma_2
# dma_3 = overlay.axi_dma_3

# Access send and receive channels for each DMA
dma_0_send = dma_0.sendchannel
dma_0_recv = dma_0.recvchannel

# dma_1_send = dma_1.sendchannel
# dma_1_recv = dma_1.recvchannel

# dma_2_send = dma_2.sendchannel
# dma_2_recv = dma_2.recvchannel

# dma_3_send = dma_3.sendchannel
# dma_3_recv = dma_3.recvchannel

In [94]:
# Allocate buffers for each DMA
input_buffer_ae_0 = allocate(shape=(19,), dtype=np.uint64)
output_buffer_ae_0 = allocate(shape=(4,), dtype=np.uint64)

input_buffer_ae_1 = allocate(shape=(19,), dtype=np.uint64)
output_buffer_ae_1 = allocate(shape=(4,), dtype=np.uint64)

input_buffer_ae_2 = allocate(shape=(19,), dtype=np.uint64)
output_buffer_ae_2 = allocate(shape=(4,), dtype=np.uint64)

input_buffer_ae_3 = allocate(shape=(19,), dtype=np.uint64)
output_buffer_ae_3 = allocate(shape=(4,), dtype=np.uint64)

In [95]:
ae_wb_dos = load_file3(ae_dos_wb)
ae_wb_port = load_file3(ae_port_wb)
ae_wb_ddos = load_file3(ae_ddos_wb)
ae_wb_patator = load_file3(ae_patator_wb)

In [96]:
# Memproses input buffer untuk classifier

# dos
buffer_values_list_dos = []
for row in hex_buffer_ae_dos:
    buffer_values_dos = [val[-4:].upper() for val in row]  # Ambil 4 digit terakhir
    buffer_values_list_dos.append(buffer_values_dos)
final_matrix_ae_dos = np.array([ae_wb_dos + list(row) for row in buffer_values_list_dos])
final_matrix_ae_dos = final_matrix_ae_dos.reshape(hex_buffer_ae_dos.shape[0], 19)
# print(final_matrix_ae_dos)

# portscan
buffer_values_list_port = []
for row in hex_buffer_ae_port:
    buffer_values_port = [val[-4:].upper() for val in row]  # Ambil 4 digit terakhir
    buffer_values_list_port.append(buffer_values_port)
final_matrix_ae_port = np.array([ae_wb_port + list(row) for row in buffer_values_list_port])
final_matrix_ae_port = final_matrix_ae_port.reshape(hex_buffer_ae_port.shape[0], 19)
# print(final_matrix_ae_port)

# ddos
buffer_values_list_ddos = []
for row in hex_buffer_ae_ddos:
    buffer_values_ddos = [val[-4:].upper() for val in row]  # Ambil 4 digit terakhir
    buffer_values_list_ddos.append(buffer_values_ddos)
final_matrix_ae_ddos = np.array([ae_wb_ddos + list(row) for row in buffer_values_list_ddos])
final_matrix_ae_ddos = final_matrix_ae_ddos.reshape(hex_buffer_ae_ddos.shape[0], 19)
# print(final_matrix_ae_ddos)

# patator
buffer_values_list_patator = []
for row in hex_buffer_ae_patator:
    buffer_values_patator = [val[-4:].upper() for val in row]  # Ambil 4 digit terakhir
    buffer_values_list_patator.append(buffer_values_patator)
final_matrix_ae_patator = np.array([ae_wb_patator + list(row) for row in buffer_values_list_patator])
final_matrix_ae_patator = final_matrix_ae_patator.reshape(hex_buffer_ae_patator.shape[0], 19)
# print(final_matrix_ae_patator)
 

In [97]:
# Alokasi buffer
input_buffer_class = allocate(shape=(19,), dtype=np.uint64)
output_buffer_class_2000 = allocate(shape=(4,), dtype=np.uint64)
output_buffer_class_4000 = allocate(shape=(4,), dtype=np.uint64)

output_history_2000 = []
output_history_4000 = []

In [98]:
# Loop pertama untuk DOS, PORT, dan PATATOR (2000 baris)
num_rows_dos = final_matrix_ae_dos.shape[0]  # 2000
print(num_rows_dos)
for i in range(num_rows_dos):
    for j in range(19):
        dos_val = int(final_matrix_ae_dos[i][j], 16)   # 16-bit (bit 63-48)
        port_val = int(final_matrix_ae_port[i][j], 16)  # 16-bit (bit 47-32)
        patator_val = int(final_matrix_ae_patator[i][j], 16)  # 16-bit (bit 15-0)

        # Gabungkan ke dalam satu nilai 64-bit
        input_buffer_class[j] = (dos_val << 48) | (port_val << 32) | (0 << 16) | patator_val  # ddos kosong (0)
    
    # Kirim ke DMA
    dma_0_send.transfer(input_buffer_class)
    dma_0_send.wait()
    dma_0_recv.transfer(output_buffer_class_2000)
    dma_0_recv.wait()
    
    # Simpan hasilnya
    output_history_2000.append(output_buffer_class_2000.copy())

# Loop kedua untuk DDOS (4000 baris)
num_rows_ddos = final_matrix_ae_ddos.shape[0]  # 4000
print(num_rows_ddos)
for i in range(num_rows_ddos):
    for j in range(19):
        ddos_val = int(final_matrix_ae_ddos[i][j], 16)  # 16-bit (bit 31-16)

        # Gabungkan ke dalam satu nilai 64-bit (hanya bagian DDOS yang diproses)
        input_buffer_class[j] = (0 << 48) | (0 << 32) | (ddos_val << 16) | 0  # dos, port, patator kosong (0)
    
    # Kirim ke DMA
    dma_0_send.transfer(input_buffer_class)
    dma_0_send.wait()
    dma_0_recv.transfer(output_buffer_class_4000)
    dma_0_recv.wait()
    
    # Simpan hasilnya
    output_history_4000.append(output_buffer_class_4000.copy())

# Tampilkan 5 baris pertama
print("Output History 2000 (5 baris pertama):")
print(output_history_2000[:5]) 
# [:5, :]
print("\nOutput History 4000 (5 baris pertama):")
print(output_history_4000[:5])

2000
4000
Output History 2000 (5 baris pertama):
[PynqBuffer([ 14355223812243456,                  0,  86694292826882048,
            311874274195406848], dtype=uint64), PynqBuffer([ 14355223812243456,                  0,  86694292826882048,
            311874274195406848], dtype=uint64), PynqBuffer([ 15199648742375424,                  0,  86694292826882048,
            311874274195406848], dtype=uint64), PynqBuffer([ 15199648742375424,                  0,  86694292826882048,
            311874274195406848], dtype=uint64), PynqBuffer([  14918173765664768,                   0,   86694292826882048,
            1448470230153035776], dtype=uint64)]

Output History 4000 (5 baris pertama):
[PynqBuffer([                   0,                    0,
              307089199591325696, 18446181123756130304], dtype=uint64), PynqBuffer([                   0,                    0,
              307089199591325696, 18446181123756130304], dtype=uint64), PynqBuffer([                   0,                

In [99]:
# Ubah ke numpy array
output_history_2000 = np.array(output_history_2000, dtype=np.uint64)
output_history_4000 = np.array(output_history_4000, dtype=np.uint64)

# Ambil 16-bit MSB dari setiap elemen
output_msb_2000 = (output_history_2000 >> 48) & 0xFFFF
output_msb_4000 = (output_history_4000 >> 48) & 0xFFFF

# print(output_msb_2000[:5])
# print(output_msb_4000[:5])

# Cetak hasil dalam format heksadesimal
print(np.array2string(output_msb_2000[:5, :], formatter={'int': lambda x: f"0x{x:04X}"}))
print(np.array2string(output_msb_4000[:5, :], formatter={'int': lambda x: f"0x{x:04X}"}))
print(np.array2string(output_msb_2000[1545:1560], formatter={'int': lambda x: f"0x{x:04X}"}))
print(np.array2string(output_msb_4000[1600:1620], formatter={'int': lambda x: f"0x{x:04X}"}))

[[0x0033 0x0000 0x0134 0x0454]
 [0x0033 0x0000 0x0134 0x0454]
 [0x0036 0x0000 0x0134 0x0454]
 [0x0036 0x0000 0x0134 0x0454]
 [0x0035 0x0000 0x0134 0x141A]]
[[0x0000 0x0000 0x0443 0xFFFE]
 [0x0000 0x0000 0x0443 0xFFFE]
 [0x0000 0x0000 0x09A6 0xFFFE]
 [0x0000 0x0000 0x09A6 0xFFFE]
 [0x0000 0x0000 0x0FAF 0xFFFE]]
[[0x0000 0x0000 0x0134 0x18DB]
 [0xFFFE 0x0000 0x0134 0x0000]
 [0xFFFE 0x0000 0x0134 0x0000]
 [0xFFFF 0x0000 0x0134 0x0000]
 [0xFFFF 0x0000 0x0134 0x0000]
 [0xFFFF 0x0000 0x0134 0x0000]
 [0xFFFF 0x0000 0x0134 0x0000]
 [0x0004 0x0000 0x0134 0x0000]
 [0x0004 0x0000 0x0134 0x0000]
 [0x0019 0x0000 0x0134 0x0000]
 [0x0019 0x0000 0x0134 0x0000]
 [0x0024 0xFFFF 0x0134 0x0000]
 [0x0024 0xFFFF 0x0134 0x0000]
 [0x002A 0x02F8 0x0134 0x0000]
 [0x002A 0x02F8 0x0134 0x0000]]
[[0x0000 0x0000 0x0825 0xFFFE]
 [0x0000 0x0000 0x0825 0xFFFE]
 [0x0000 0x0000 0x0836 0xFFFE]
 [0x0000 0x0000 0x0836 0xFFFE]
 [0x0000 0x0000 0x0911 0xFFFE]
 [0x0000 0x0000 0x0911 0xFFFE]
 [0x0000 0x0000 0x0A4C 0xFFFE]
 [0x0

In [100]:
# Konversi output_msb ke desimal
output_dec_2000 = np.vectorize(lambda x: fixed_point_to_decimal(f"{x:04X}"))(output_msb_2000)
output_dec_4000 = np.vectorize(lambda x: fixed_point_to_decimal(f"{x:04X}"))(output_msb_4000)

print(output_dec_2000[:5])
print(output_dec_4000[:5])

[[0.04980469 0.         0.30078125 1.08203125]
 [0.04980469 0.         0.30078125 1.08203125]
 [0.05273438 0.         0.30078125 1.08203125]
 [0.05273438 0.         0.30078125 1.08203125]
 [0.05175781 0.         0.30078125 5.02539062]]
[[ 0.00000000e+00  0.00000000e+00  1.06542969e+00 -1.95312500e-03]
 [ 0.00000000e+00  0.00000000e+00  1.06542969e+00 -1.95312500e-03]
 [ 0.00000000e+00  0.00000000e+00  2.41210938e+00 -1.95312500e-03]
 [ 0.00000000e+00  0.00000000e+00  2.41210938e+00 -1.95312500e-03]
 [ 0.00000000e+00  0.00000000e+00  3.92089844e+00 -1.95312500e-03]]


In [56]:
print(output_dec_2000[101:200])
# print(output_dec_4000[22:34])
print("")
print(output_dec_2000[1101:1200])
# print(output_dec_4000[2299:2319])

[[-1.95312500e-03  0.00000000e+00  3.00781250e-01  0.00000000e+00]
 [-1.95312500e-03  0.00000000e+00  3.00781250e-01  0.00000000e+00]
 [-1.95312500e-03  0.00000000e+00  3.00781250e-01  0.00000000e+00]
 [ 1.66015625e-02  0.00000000e+00  3.00781250e-01  0.00000000e+00]
 [ 1.66015625e-02  0.00000000e+00  3.00781250e-01  0.00000000e+00]
 [ 2.53906250e-02  0.00000000e+00  3.00781250e-01  0.00000000e+00]
 [ 2.53906250e-02  0.00000000e+00  3.00781250e-01  0.00000000e+00]
 [ 2.53906250e-02  0.00000000e+00  3.00781250e-01  0.00000000e+00]
 [ 2.53906250e-02  0.00000000e+00  3.00781250e-01  0.00000000e+00]
 [ 2.50976562e-01 -1.95312500e-03  3.00781250e-01  0.00000000e+00]
 [ 2.50976562e-01 -1.95312500e-03  3.00781250e-01  0.00000000e+00]
 [ 2.54882812e-01  1.58203125e-01  3.00781250e-01  0.00000000e+00]
 [ 2.54882812e-01  1.58203125e-01  3.00781250e-01  0.00000000e+00]
 [ 4.81445312e-01  2.33984375e+00  3.00781250e-01  0.00000000e+00]
 [ 4.81445312e-01  2.33984375e+00  3.00781250e-01  0.00000000e

In [101]:
del output_history_2000
del output_history_4000
del output_msb_2000
del output_msb_4000

In [102]:
del final_matrix_ae_dos
del final_matrix_ae_port
del final_matrix_ae_ddos
del final_matrix_ae_patator

In [103]:
# Free buffers

del input_buffer_ae_0
del output_buffer_ae_0

del input_buffer_ae_1
del output_buffer_ae_1

del input_buffer_ae_2
del output_buffer_ae_2

del input_buffer_ae_3
del output_buffer_ae_3

del input_buffer_class
del output_buffer_class_2000
del output_buffer_class_4000

del overlay  # Menghapus overlay sebelumnya

In [104]:
# Pisahkan setiap kolom menjadi array terpisah
predict_dos = output_dec_2000[:, 0]      # Kolom pertama
predict_port = output_dec_2000[:, 1]     # Kolom kedua
predict_ddos = output_dec_4000[:, 2]     # Kolom ketiga
predict_patator = output_dec_2000[:, 3]  # Kolom keempat

In [105]:
for i in range(4000):
    if i < 1000:  # Indeks 0-999
        if predict_dos[i] < 0:
            predict_dos[i] = 0
        if predict_port[i] < 0:
            predict_port[i] = 0
        if predict_ddos[i] < 0:
            predict_ddos[i] = 0
        if predict_patator[i] < 0:
            predict_patator[i] = 0
    elif 1000 <= i < 2000: 
        if predict_dos[i] >= 0:
            predict_dos[i] = 1
        if predict_port[i] >= 0:
            predict_port[i] = 1
        if predict_ddos[i] < 0:
            predict_ddos[i] = 0
        if predict_patator[i] >= 0:
            predict_patator[i] = 1
    else:
        if predict_ddos[i] >= 0:
            predict_ddos[i] = 1

In [106]:
# Buat array dengan ukuran yang sesuai
label_dos = np.zeros(2000, dtype=np.uint16)
label_port = np.zeros(2000, dtype=np.uint16)
label_ddos = np.zeros(4000, dtype=np.uint16)
label_patator = np.zeros(2000, dtype=np.uint16)

# Isi baris 1001-2000 dengan 1 untuk label_dos, label_port, dan label_patator
label_dos[1000:2000] = 1
label_port[1000:2000] = 1
label_patator[1000:2000] = 1

# Isi baris 2001-4000 dengan 1 untuk label_ddos
label_ddos[2000:4000] = 1

# Cetak hasil (opsional)
# print("label_dos:\n", label_dos)
# print("label_port:\n", label_port)
# print("label_ddos:\n", label_ddos)
# print("label_patator:\n", label_patator)

In [110]:
# Konversi prediksi ke biner (anggap threshold 0.5 untuk VAE)

threshold = 0.5
predict_dos_bin = (predict_dos >= threshold).astype(int)
predict_port_bin = (predict_port >= threshold).astype(int)
predict_ddos_bin = (predict_ddos >= threshold).astype(int)
predict_patator_bin = (predict_patator >= threshold).astype(int)

predict_dos_bin = predict_dos_bin[:2000]
predict_port_bin = predict_port_bin[:2000]
predict_patator_bin = predict_patator_bin[:2000]

In [111]:
print("Shape label_dos:", label_dos.shape)
print("Shape predict_dos_bin:", predict_dos_bin.shape)

print("Shape label_port:", label_port.shape)
print("Shape predict_port_bin:", predict_port_bin.shape)

print("Shape label_ddos:", label_ddos.shape)
print("Shape predict_ddos_bin:", predict_ddos_bin.shape)

print("Shape label_patator:", label_patator.shape)
print("Shape predict_patator_bin:", predict_patator_bin.shape)


Shape label_dos: (2000,)
Shape predict_dos_bin: (2000,)
Shape label_port: (2000,)
Shape predict_port_bin: (2000,)
Shape label_ddos: (4000,)
Shape predict_ddos_bin: (4000,)
Shape label_patator: (2000,)
Shape predict_patator_bin: (2000,)


In [112]:
# Fungsi untuk menghitung metrik
def calculate_metrics(y_true, y_pred):
    tp = np.sum((y_pred == 1) & (y_true == 1))  # True Positives
    tn = np.sum((y_pred == 0) & (y_true == 0))  # True Negatives
    fp = np.sum((y_pred == 1) & (y_true == 0))  # False Positives
    fn = np.sum((y_pred == 0) & (y_true == 1))  # False Negatives
    
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return accuracy, precision, recall, f1_score

# Hitung metrik untuk setiap label
metrics_dos = calculate_metrics(label_dos, predict_dos_bin)
metrics_port = calculate_metrics(label_port, predict_port_bin)
metrics_ddos = calculate_metrics(label_ddos, predict_ddos_bin)
metrics_patator = calculate_metrics(label_patator, predict_patator_bin)

# Tampilkan hasil
print("DOS Metrics:", metrics_dos)
print("PORT Metrics:", metrics_port)
print("DDOS Metrics:", metrics_ddos)
print("PATATOR Metrics:", metrics_patator)

DOS Metrics: (0.858, 0.8003355704697986, 0.954, 0.8704379562043795)
PORT Metrics: (0.873, 0.8123953098827471, 0.97, 0.8842297174111212)
DDOS Metrics: (0.8645, 0.8060453400503779, 0.96, 0.8763121862163396)
PATATOR Metrics: (0.864, 0.8043478260869565, 0.962, 0.8761384335154827)


In [113]:
del predict_dos, predict_port, predict_ddos, predict_patator
del label_dos, label_port, label_ddos, label_patator
del predict_dos_bin, predict_port_bin, predict_ddos_bin, predict_patator_bin
del threshold, calculate_metrics
del metrics_dos, metrics_port, metrics_ddos, metrics_patator
del output_dec_2000
del output_dec_4000